In [ ]:
import numpy as np
from hdmm.workload import MarginalsGram
from scipy import sparse
from scipy.sparse.linalg import spsolve_triangular

In [ ]:
dom = (2,3,4)
w = np.array([1,1,1,0,1,0,0,0])

WtW = MarginalsGram(dom, w)

In [ ]:
def loss0(theta):
    AtA = MarginalsGram(dom, theta**2).dense_matrix()
    AtA1 = np.linalg.pinv(AtA)
    X = WtW.dense_matrix()
    return np.trace(X @ AtA1)

def loss1(theta):
    theta2 = theta**2
    X = WtW._Xmatrix(theta2)[0]
    D = sparse.diags(X.dot(np.ones_like(theta))==0, dtype=float)
    phi = spsolve_triangular(X+D, w, lower=False)
    ans = np.sum(phi) * np.prod(dom)
    return ans

theta = np.array([0.25,0.25,0,0.1,0.4,0,0,0])

print(loss0(theta), loss1(theta))

In [ ]:
def loss_and_grad(theta):
    n, d = np.prod(WtW.domain), len(WtW.domain)
    A = np.arange(2**d)
    mult = WtW._mult
    weights = WtW.weights
    ones = np.ones_like(theta)
    
    delta = np.sum(theta)**2
    ddelta = 2*np.sum(theta)
    theta2 = theta**2
    
    X, XT = WtW._Xmatrix(theta2)
    D = sparse.diags(X.dot(np.ones_like(theta))==0, dtype=float)
    phi = spsolve_triangular(X+D, w, lower=False)
    ans = np.sum(phi) * n
    
    dXvect = -spsolve_triangular(XT+D, ones*n, lower=True)
    # dX = outer(dXvect, phi)
    dtheta2 = np.array([np.dot(dXvect[A&b]*phi, mult[A|b]) for b in range(2**d)])
    dtheta = 2*theta*dtheta2
    return delta*ans, delta*dtheta + ddelta*ans

theta = np.array([0.25,0.25,0,0.1,0.4,0,0,0])
theta = np.random.rand(8); theta[-1] = 0
f0, g0 = loss_and_grad(theta)
g1 = np.zeros(8)

for i in range(8):
    theta[i] -= 0.0001
    f1, _ = loss_and_grad(theta)
    theta[i] += 0.0002
    f2, _ = loss_and_grad(theta)
    theta[i] -= 0.0001
    g1[i] = (f2 - f1) / 0.0002

print(g0)
print(g1)
        

In [4]:
from ektelo.hdmm import templates
from ektelo import workload
import numpy as np
#%load_ext autoreload
#%autoreload 2

W = workload.DimKMarginals((2,3,4), 2)
temp = templates.Marginals((2,3,4), approx=True)

temp._set_workload(W)

theta = np.random.rand(8); theta[-1] = 0
f0, g0 = temp._loss_and_grad(theta)
g1 = np.zeros(8)

for i in range(8):
    theta[i] -= 0.0001
    f1, _ = temp._loss_and_grad(theta)
    theta[i] += 0.0002
    f2, _ = temp._loss_and_grad(theta)
    theta[i] -= 0.0001
    g1[i] = (f2 - f1) / 0.0002

print(g0)
print(g1)

[   65.91583695    53.86791272    54.18725627    21.55158659
    63.78023997 -2654.89930246   -20.82259202  -881.11188754]
[   65.91583695    53.86791259    54.18725606    21.55158609
    63.78023995 -2654.90566358   -20.82259985  -881.11211509]
